

#### bring case list

#### pre-process 판시사항, 판결요지, 판례내용 
#### [가-힣만 남겨두기]
#### 판결요지, 판례내용을 list of sentences로 만들기
#### kkma tokenizer를 token list만 제공하는 형태로 post_fn을 만들어서 넘기기
#### keyword와 keysentence를 구하기
#### keyword와 keysentence 를 text 기준 위치를 구하는 함수를 만들기
#### substring 비교식으로 해야할 것

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 17.5 MB/s 
     |████████████████████████████████| 465 kB 82.2 MB/s 


In [ ]:
from konlpy.tag import Kkma 
import pandas as pd
import numpy as np
import re

In [ ]:
!git clone https://github.com/Mubbit/Level1_w_data.git

Cloning into 'Level1_w_data'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 137 (delta 2), reused 7 (delta 2), pack-reused 126
Receiving objects: 100% (137/137), 29.30 MiB | 10.46 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [ ]:
!rm -r textrank
!git clone https://github.com/Mubbit/textrank.git
!git pull

Cloning into 'textrank'...
remote: Enumerating objects: 275, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 275 (delta 18), reused 11 (delta 1), pack-reused 225
Receiving objects: 100% (275/275), 1.60 MiB | 22.76 MiB/s, done.
Resolving deltas: 100% (150/150), done.
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
!pip install textrank

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for textrank: filename=textrank-0.1.0-py3-none-any.whl size=3419 sha256=04248ad568c2162d192cd998fff774c8627d5b13e08f4782fdad929f8c54dc48
  Stored in directory: /root/.cache/pip/wheels/47/5f/18/15997fca2df9b310c344b8598b750645a689e0aa0ba82f3289
Successfully built textrank


/content/textrank/textrank/summarizer.py

In [ ]:
!ls textrank/

data  MIT-LICENSE.txt  README.md  run_textrank.ipynb  setup.py	textrank


In [ ]:
import sys
sys.path.append('textrank/textrank/')
from summarizer import KeywordSummarizer,KeysentenceSummarizer

class PreProcess:#1개 이상의 판례를 대상으로 진행한다. 
  def __init__(self,case,patterns=re.compile('[^(가-힣 .)]')):
    self.ppsh=list(case.loc[:,'holding'].values) #list of paragraphs
    self.pgyj=list(case.loc[:,'summary'].values) #list of paragraphs
    self.prny=list(case.loc[:,'prny'].values) #list of paragraphs
    self.filter_all(patterns)
  def filter(self,li,pattern=re.compile('[^(가-힣 .)]')):    #하나만 들어오더라도 []에 넣어주므로 동질적으로 진행 가능하다.
      new_series=[]
      for paragraph in li:
          new_series.append(re.sub(pattern,' ',paragraph))
      return new_series 
  def filter_all(self,patterns=re.compile('[^(가-힣 .)]')):
      if isinstance(patterns,list):
          print('not implemented for multiple pattern filtering')
      else:
        if isinstance(self.ppsh,list):
          self.filtered_df=pd.DataFrame({
          'holding':self.filter(self.ppsh),
          'summary':self.filter(self.pgyj),
          'prny':self.filter(self.prny)})
        else:
          self.filtered_df=pd.DataFrame({
          'holding':self.filter(self.ppsh),
          'summary':self.filter(self.pgyj),
          'prny':self.filter(self.prny)},index=[0])


class InferTextRank:
  def __init__(self):
    self.kkma=Kkma().nouns
    self.keyword_extractor=KeywordSummarizer(tokenize=lambda x: self.kkma(x),window=5,verbose=False)
    self.keysentence_extractor=KeysentenceSummarizer(tokenize=lambda x:self.kkam(x),verbose=False)
  def textrank_sent(self,paragraph):
    keysentences=self.keysentence_extractor.summarize(paragraph,topk=2)
    sents=[]
    for sent,rank in keysentences:
      sents.append(sent)
    return sents
  def textrank_word(self,paragraph,limit=10):
    #paragraph: list of sents
    keywords=self.keyword_extractor.summarize(paragraph,topk=limit)
    words=[]
    count=0
    for word,rank in keywords:
      if count<10:
        words.append(word)
      else:
        break
      count+=1
    return words
  
      
    

In [ ]:
case=pd.read_csv('Level1_w_data/case_list.csv')
case=case.rename(columns={'판례내용':'prny'})

In [ ]:
case.columns

Index(['index', 'datanumber', 'title', 'casenumber', 'date', 'verdicttype',
       'holding', 'summary', 'jmreference', 'prreference', 'prny'],
      dtype='object')

In [ ]:
pre_process=PreProcess(case)
pre_process.filter_all()# df of three columns '판시사항' '판결요지' 판례내용'
pssh_orig=pre_process.filtered_df['holding']
pgyj_orig=pre_process.filtered_df['summary']
prny_orig=pre_process.filtered_df['prny']

판시사항이 . 으로 문장이 구분되지 않고 '여부'로 구분되는 것이 특징

In [ ]:
case.head(5)

,index,datanumber,title,casenumber,date,verdicttype,holding,summary,jmreference,prreference,prny
0,0,230961,주주총회소집허가,2022마5372,20220907,결정,소수주주가 상법 제366조에 따라 임시총회 소집에 관한 법원의 허가를 신청할 때...,일반적으로 주주총회는 이사회의 결의로 소집하지만(상법 제362조)$ 예외적으로 ...,상법 제361조$ 제362조$ 제366조$ 제385조 제1항$ 제389조$ 민사소송...,대법원 2022. 4. 19. 자 2022그501 결정(공2022상$ 1013),【신청인$ 재항고인】 신청인 (소송대리인 변호사 오원근)【사건본인$ 상대방】 주식회...
1,1,230971,소유권이전등기,2022다217117,20220907,판결,[1] 수임인이 위임인을 위하여 자기 명의로 취득한 권리를 위임인에게 이전하여야...,"[1] 민법 제684조 제2항은 ""수임인이 위임인을 위하여 자기의 명의로 취득한...",[1] 민법 제684조 / [2] 민법 제684조,[1] 대법원 2007. 2. 8. 선고 2004다64432 판결(공2007상$ ...,【원고$ 상고인 겸 피상고인】 하동지구개발사업단 주식회사의 파산관재인 (소송대리인 ...
2,2,230967,폭력행위등처벌에관한법률위반(단체등의구성·활동)·폭력행위등처벌에관한법률위반(단체등의공...,2022도6993,20220907,판결,[1] 공소장변경이 허용되는 범위 / 공소사실의 동일성이 인정되지 않는 범죄사실...,[1] 공소장변경은 공소사실의 동일성이 인정되는 범위 내에서만 허용되고$ 공소사...,[1] 형사소송법 제298조 제1항 / [2] 폭력행위 등 처벌에 관한 법률 제4...,[1] 대법원 1994. 3. 22. 선고 93도2080 전원합의체 판결(공199...,【피 고 인】 피고인 1 외 1인【상 고 인】 피고인들【변 호 인】 변호사 현희철 ...
3,3,230957,연차수당지급,2022다245419,20220907,판결,1년을 초과하되 2년 이하의 기간 동안 근로를 제공한 근로자에게 부여될 수 있는...,근로기준법에 따르면$ 사용자는 1년간 80% 이상 출근한 근로자에게 15일의 연...,근로기준법 제60조 제1항$ 제2항,대법원 2018. 6. 28. 선고 2016다48297 판결(공2018하$ 1435...,【원고$ 상고인】 성창산업개발 주식회사【피고$ 피상고인】 재단법인 오송첨단의료산업진...
4,4,230981,중재판정의집행,2020마5970,20220907,결정,중재요청서 등의 서면 통지에 있어서 중재법 제4조 제3항의 발송에 의한 통지가 ...,중재절차는 당사자 간에 다른 합의가 없는 한 피신청인이 중재요청서를 받은 날부터...,중재법 제4조$ 제22조 제1항$ 제36조 제2항 제1호 (나)목$ 제38조 제1호...,-,【신청인$재항고인】 주식회사 포스코아이씨티 (소송대리인 법무법인(유한) 화우 담당변...


In [ ]:

print(pssh_orig.head(3))
print(pgyj_orig.head(3))
print(prny_orig.head(3))

0      소수주주가 상법 제   조에 따라 임시총회 소집에 관한 법원의 허가를 신청할 때...
1          수임인이 위임인을 위하여 자기 명의로 취득한 권리를 위임인에게 이전하여야...
2          공소장변경이 허용되는 범위   공소사실의 동일성이 인정되지 않는 범죄사실...
Name: holding, dtype: object
0      일반적으로 주주총회는 이사회의 결의로 소집하지만(상법 제   조)  예외적으로 ...
1          민법 제   조 제 항은  수임인이 위임인을 위하여 자기의 명의로 취득한...
2          공소장변경은 공소사실의 동일성이 인정되는 범위 내에서만 허용되고  공소사...
Name: summary, dtype: object
0     신청인  재항고인  신청인 (소송대리인 변호사 오원근) 사건본인  상대방  주식회...
1     원고  상고인 겸 피상고인  하동지구개발사업단 주식회사의 파산관재인 (소송대리인 ...
2     피 고 인  피고인   외  인 상 고 인  피고인들 변 호 인  변호사 현희철 ...
Name: prny, dtype: object


In [ ]:
class Highlight:
  def __init__(self,keyword_list,paragraph):
    self.keyword=keyword_list
    self.paragraph=paragraph
  def highlight(self,element):
    if element in self.keyword:
      if len(element)!=0:
        return '<highlighted>'+element+'</highlighted>'
    else:
      once=False 
      fr=''
      for i in self.keyword:
        if element in i:
          once=True
          fr=i
      if once==True:
        return '<highlighted>'+fr+'</highlighted>'
    return element

def map_and_combine(paragraph,keywords,keysentences):
  #print(keysentences)
  highlighter=Highlight(keywords,paragraph)
  res=list(map(highlighter.highlight,paragraph.split()))
  ret=''
  for w in res:
    ret+=' '+w
  return ret[1:]

test_keywords=['apple','banana']
test_paragraph='The monkey fell off the tree because it mistook an apple tree as a banana tree and tried to climb it the way it would upon a banana tree'
print(map_and_combine(test_paragraph,test_keywords,['tried_to_climb']))


The monkey fell off the tree because it mistook <highlighted>banana</highlighted> <highlighted>apple</highlighted> tree as <highlighted>banana</highlighted> <highlighted>banana</highlighted> tree and tried to climb it the way it would upon <highlighted>banana</highlighted> <highlighted>banana</highlighted> tree


#Infer

In [ ]:
for row in pgyj_orig.values:
  if len(row.split('.'))>2:
    print('exist')
  if '.' in row:
    print('yes exist')

exist
yes exist
exist
yes exist
exist
yes exist
exist
yes exist
exist
yes exist
exist
yes exist
exist
yes exist
exist
yes exist
yes exist
exist
yes exist
exist
yes exist
exist
yes exist
exist
yes exist
exist
yes exist
exist
yes exist
exist
yes exist
exist
yes exist
exist
yes exist
exist
yes exist
exist
yes exist


In [ ]:
for row in pssh_orig.values:
  if len(row.split('여부'))>2:
    print('exist')
  if '.' in row:
    print('yes exist')

yes exist
yes exist
yes exist
yes exist
yes exist
yes exist
exist
yes exist
yes exist
yes exist
yes exist
yes exist
yes exist
yes exist
yes exist
yes exist
yes exist
exist
yes exist
yes exist
yes exist
yes exist
yes exist
yes exist
yes exist
yes exist
yes exist
exist
yes exist
yes exist
yes exist
yes exist
yes exist
yes exist
yes exist
yes exist
yes exist
yes exist
exist
yes exist
yes exist
yes exist


테스트 완료 후

In [ ]:

pssh=[]
pgyj=[]
prny=[]
def remove_empty(sent_list):
  result=[]
  for l in sent_list: 
    if l!='' and l !=' ':
      result.append(l)
  return result

def return_list(df,keyword,limit=10):
  tt=InferTextRank()
  word_list=[]
  keysentence_list=[]
  new_paragraph_list=[]
  i=0
  for paragraph in df[keyword]:
    
    if keyword=='summary':
      #print('working?')
      sent_list=list(map(str.strip,paragraph.split('여부')))
    else:
      sent_list=list(map(str.strip,paragraph.split('.')))
    sent_list=remove_empty(sent_list)
    print('sent list for',keyword,'has',len(sent_list))
    if len(sent_list)>1:
      try:
        words=tt.textrank_word(sent_list,limit=limit)
        word_list.append(words)

        keysents=tt.textrank_sent(sent_list)
        keysentence_list.append(keysents)

        #paragraph내 words의 위치를 찾아 다 <태그> 붙여주기
        tmp_paragraph=paragraph
        '''for word in words:
          tmp_paragraph=re.sub(word,'<highlighted>'+word+'</highlighted>',tmp_paragraph)
        '''
        new_paragraph=map_and_combine(tmp_paragraph,words,keysentence_list)
        new_paragraph_list.append(new_paragraph)
      except:
        if i>limit:
          i+=1
          print(len(sent_list))
          print(sent_list)
          break
        word_list.append(['None']*limit)   
        new_paragraph_list.append(paragraph) 
    else:
      word_list.append(['None']*limit)
      new_paragraph_list.append(paragraph)

    
  return word_list,keysentence_list,new_paragraph_list

pssh,pssh_keysent,pssh_replace=return_list(pre_process.filtered_df,'holding')
pgyj,pgyj_keysent,pgyj_replace=return_list(pre_process.filtered_df,'summary')
prny,prny_keysent,prny_replace=return_list(pre_process.filtered_df,'prny')

In [ ]:
print(pssh_keysent)
print(pgyj_keysent)
print(prny_keysent)

[]
[]
[]


In [ ]:
if '.' in pssh_replace[0]:# or '.' in prny_replace[0]:
  print('yes')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
def return_sents(paragraph,keyword):
    if keyword=='summary':
      #print('working?')
      sent_list=list(map(str.strip,paragraph.split('여부')))
    else:
      sent_list=list(map(str.strip,paragraph.split('.')))
    sent_list=remove_empty(sent_list)
    return sent_list
corpus=return_sents(prny_replace[0],'prny')
'''corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]'''
kkma=Kkma().nouns
stopwords=['채','이하','이유','사건','사실','주', '주문', '직', '호', '현', '후', '지', '조', '미', '과', '그', '이', '이때', '위', '유', '을', '의', '수', '소', '악', '별', '때', '등', '라', '문', '미', '법', '김', '그', '경우', '기각', '와', '함께', '그래서', '여부', '가' ]
vectorizer = TfidfVectorizer(tokenizer=lambda x: kkma(x),stop_words=stopwords)
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out())
print(X.shape)

['견제' '결의' '결의사항' '결정' '고등' '고법' '공익권' '관여' '권한' '규정' '기록' '기재' '기회' '김선수'
 '노태' '노태악' '대리인' '대법관' '대법원' '대전' '대전고등법원' '대전고법' '대표' '대표이사' '대표이사직'
 '대표이사직도' '목록' '목적' '목적사항' '박정' '발행' '발행주식총수' '법리' '법원' '변경' '변호사' '별지'
 '보호' '본인' '부분' '부의' '분의' '불일치' '사건본인' '사항' '상대방' '상법' '상실' '생략' '생략대법관'
 '서면' '석명' '석명의무' '선수' '선임' '소명' '소명자료' '소송' '소송대리인' '소수' '소수주주' '소외'
 '소외인' '소외인의' '소집' '소집청구' '수정' '신청' '신청인' '심리' '안건' '예외적' '오' '오경' '오경미'
 '오원근' '외인' '원근' '원심' '원심결정' '원심법원' '원심판결' '의견' '의무' '의미' '의사' '의안' '이사'
 '이사직' '이사회' '이사회의' '이상' '이익' '인의' '일반적' '일부' '일치' '임시' '임시주주총회' '임시총회'
 '임시총회소집청구권' '임시총회소집청구서' '임시총회소집허가' '자' '자동적' '자료' '자신' '잘못' '재판' '재판장'
 '재항고' '재항고이유' '절차' '점' '정관' '정확' '제' '제안' '제출' '조치' '주식' '주식회사' '주심' '주장'
 '주주' '주주총회' '중' '즉시' '즉시항고장' '지배' '지배주주' '지적' '지지' '지체' '직도' '직접' '진술'
 '진행' '참조' '청구' '청구권' '청구서' '총수' '총회' '총회소집' '출한' '취지' '코리아' '파기' '판결'
 '판단' '필요' '하고자' '하나' '한편' '항고인' '항고장' '해당' '해임' '허가' '환송' '회사' '회의' '후임']
(21, 164)


In [ ]:
new_pssh_replace=[]
new_pgyj_replace=[]
for p,pg in zip(pssh_replace,pgyj_replace):
  t=map_and_combine(p,vectorizer.get_feature_names_out(),[])
  j=map_and_combine(pg,vectorizer.get_feature_names_out(),[])
  new_pssh_replace.append(t)
  new_pgyj_replace.append(j)
  if '<highlighted>' in j:
    print('yay')

yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay


In [ ]:
highlighted_passage=pd.DataFrame([(pssh,pgyj) for pssh,pgyj in zip(new_pssh_replace,new_pgyj_replace)],columns=['jmreference_textrank','summary_textrank'])
case_list_new=pd.concat([case,highlighted_passage],axis=1)
case_list_new.drop(columns=['date','verdicttype'])
highlighted_passage.to_csv('highlighted_passage.csv',index=False)
case_list_new.to_csv('case_list_with_textrank.csv',index=False)

#REST

In [ ]:
import numpy as np
p1=pssh.copy()
p2=pgyj.copy()
p3=prny.copy()

np.save('pssh.npy',np.array(p1))
np.save('pgyj.npy',np.array(p2))
np.save('prny.npy',np.array(p3))


In [ ]:
np.unique(np.array(p1))

In [ ]:
import pandas as pd

columns=['rank'+str(i+1) for i in range(10)]
pd.DataFrame(p1,columns=columns).to_csv('/content/pssh.csv')
pd.DataFrame(p2,columns=columns).to_csv('/content/pgyj.csv')
pd.DataFrame(p3,columns=columns).to_csv('/content/prny.csv')

In [ ]:
def insert_tag(words,paragraph):
  

In [ ]:
import os.path as osp

#need work: search가 한 개만 찾아주는 반면 해당 word는 paragraph에 여러개 있을 수 있음
class FindPlace:
  def __init__ ( self,paragraph_list,textrank_list,name='data',dir='path'):
    if len(paragraph_list)==len(textrank_list):
      print('??')
      self.dir=osp.join(dir,name)
      self.find_places(paragraph_list,textrank_list)
    else:
      print('fail')
  def find_place(self,paragraph,ranked_words,count):#하나의 paragraph에 대해
    starts=[]
    ends=[]
    for word in ranked_words:
      if isinstance(word,str) and word !='None':
        try:
          start=re.search(word, paragraph).start()
          end=start+len(word)
          starts.append(start)
          ends.append(end) 
        except:
          print(word,' is not in',paragraph)
    pd.DataFrame({'start':starts,'end':ends}).to_json(osp.join(self.dir,str(count)+'.json'))

  def find_places(self,paragraph_list,textrank_list):
    count=0
    for paragraph,ranked_text in zip(paragraph_list,textrank_list):
      self.find_place(paragraph,ranked_text,count)
      count+=1 


In [ ]:
FindPlace(pre_process.filtered_df['판시사항'],pssh,name='pssh',dir='/content/jsons/')

In [ ]:

FindPlace(pre_process.filtered_df['판결요지'],pgyj,name='pgyj',dir='/content/jsons')

In [ ]:

FindPlace(pre_process.filtered_df['판례내용'],prny,name='prny',dir='/content/jsons')

In [ ]:
FindPlace(pre_process.filtered_df['판시사항'],pssh,name='pssh',dir='\\content\\jsons\\pssh')
FindPlace(pre_process.filtered_df['판례내용'],prny,name='prny',dir='\\content\\jsons\\prny')